## Part 1

In [ ]:
import re
from bs4 import BeautifulSoup
import pandas as pd       
train = pd.read_csv("labeledTrainData.tsv", header=0, \
                    delimiter="\t", quoting=3)

In [ ]:
train.shape # 학습용 데이터수와 차수 표시

In [ ]:
train

In [ ]:
train.head()


In [ ]:
# まずは一番最初の人のレビューを見てみます。 첫번째 리뷰 문워커 영화에 대한 리뷰 긍정리뷰
print (train["review"][0])

#버그로인해 <br> 등의 문구가 들어가게 됨.

In [ ]:
# remove tag
rows = []
for t in train["review"]:
    soup = BeautifulSoup(t, "html.parser")
    for s in soup.select('br'):
        s.extract()
    rows.append(soup.get_text())
train["review"] = rows

In [ ]:
example1 = train["review"][0]

In [ ]:
# reを使った文字以外の処理 정규화표현사용해서 일반 알파벳 같지 않은 문구들은 삭제한다. 철자만 남기게 한다.
# ここからは、(?,.,/,#,",')のような文字以外のものを処理していきます。
letters_only = re.sub("[^a-zA-Z]",           #この条件に合致するものを
                      " ",                   #これ(今回は空白)に置換する
                      example1 )  #対象とするテキストデータ
print(letters_only) 

## Part 2

In [ ]:
# 前回クリーニングしたテキストデータを単語単位にします。
words = lower_case.split()               
print(words)

### 불용어 제거

In [ ]:
import nltk
nltk.download('stopwords')  
from nltk.corpus import stopwords

In [ ]:
# 試しにstopwordsを見てみましょう。
stopwords.words("english") # 도움안되는 불용어들 삭제합시다. 

In [ ]:
# それでは、文章からストップワードを除去します。
words = [w for w in words if not w in stopwords.words("english")]
print (words)

In [ ]:
def review_to_words( raw_review ):
    # 1. HTMLのタグ消し
    review_text = BeautifulSoup(raw_review).get_text() 
    #
    # 2. 文字以外(?,!等)の除去       
    letters_only = re.sub("[^a-zA-Z]", " ", review_text) 
    #
    # 3. 全ての文字を小文字に変更
    words = letters_only.lower().split()                             
    #
    # 4. ストップワードのリスト作成
    stops = set(stopwords.words("english"))                  
    # 
    # 5. ストップワードの除去
    meaningful_words = [w for w in words if not w in stops]   
    #
    # 6.　文にして結果を返す
    return( " ".join( meaningful_words ))   

In [ ]:
#running runs run -> run

In [ ]:
# これで確認します。
clean_review = review_to_words( train["review"][0] )
print(clean_review)  

In [ ]:
# 次にレビューのデータ数を確認します。
# ここで、25000個のレビューがあることが確認できます。

num_reviews = train["review"].size
num_reviews

In [ ]:
# これで全てのレビューをクリーニングします。
clean_train_reviews = []
for i in range( 0, num_reviews ):
    clean_train_reviews.append( review_to_words( train["review"][i] ) )

print(clean_train_reviews)

In [ ]:
# create bag of words with scikit-learn
print ("Creating the bag of words...\n")
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 5000)  # set the features number to 5000

In [ ]:
# 次に特徴量ベクトルを作成します。(clean_train_reviewsは上で処理をしたレビューデータ)
train_data_features = vectorizer.fit_transform(clean_train_reviews)
#ここで、250000 x 5000のデータセットが準備できたことが確認できます。
print (train_data_features.shape)

In [ ]:
# これでどの単語が特徴量として選ばれたのか確認することができます。
vocab = vectorizer.get_feature_names()
print (vocab)

In [ ]:
# 最後にcsvとして保存します。
df = pd.DataFrame(train_data_features) 
# df.columns = vocab
# df.to_csv("train_bag_of_words.csv")
df

In [ ]:
# Random Forest
print("Training the random forest...")
from sklearn.ensemble import RandomForestClassifier
# Initialize
forest = RandomForestClassifier(n_estimators = 100) 
# Traing of Random Forest
forest = forest.fit( train_data_features, train["sentiment"] )

### Test

In [ ]:
# Load "Test data"
test = pd.read_csv("testData.tsv", header=0, delimiter="\t", \
                   quoting=3) 

print(test.shape)

# Training "Test data"
clean_test_reviews = []
for i in range(0,num_reviews):
    if( (i+1) % 1000 == 0 ):
        print ("Review %d of %d\n" % (i+1, num_reviews))
    clean_review = review_to_words( test["review"][i] )
    clean_test_reviews.append( clean_review )

In [ ]:
# Transform test data to word vector
test_data_features = vectorizer.transform(clean_test_reviews)
test_data_features = test_data_features.toarray()

In [ ]:
# Predict test data with trained random forest model
result = forest.predict(test_data_features)

In [ ]:
# Display predict result (convert thr result to csv file for submit to the Kaggle)
output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )
output.to_csv( "Bag_of_Words_model.csv", index=False, quoting=3 )

In [ ]:
output